In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.impute import SimpleImputer

In [85]:
train1 = pd.read_csv(r"F:\project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},nrows=300000)

In [7]:
X_train = pd.read_csv(r"C:\Users\dbda\Desktop\project\project\x_train.csv")
y_train = pd.read_csv(r"C:\Users\dbda\Desktop\project\project\y_train.csv")

In [8]:
X_train.shape

(300000, 12)

In [9]:
y_train.shape

(300000, 1)

In [87]:
x = pd.DataFrame(train1['acoustic_data'].values)
y = pd.DataFrame(train1['time_to_failure'].values)
#y = pd.DataFrame(y)
    
y_train['time_to_failure'] = y
#print(y_train.head())
        
X_train['ave'] = x.rolling(window=3).mean()
X_train['std'] = x.rolling(window=3).std()
X_train['max'] = x.rolling(window=3).max()
X_train['min'] = x.rolling(window=3).min()
X_train['median'] = x.rolling(window=3).median()
X_train['var'] = x.rolling(window=3).var()
X_train['sum'] = x.rolling(window=3).sum()
X_train['cov'] = train1['acoustic_data'].rolling(window=3).cov()
X_train['quantile_1'] = x.rolling(window=3).quantile(0.25)
X_train['quantile_2'] = x.rolling(window=3).quantile(0.50)
X_train['quantile_3'] = x.rolling(window=3).quantile(0.75)

stepsize = np.diff(train1['acoustic_data'])
train1 = train1.drop(train1.index[len(train1)-1])
train1["stepsize"] = stepsize
X_train['stepsize'] = train1["stepsize"]
print(X_train)


             ave       std   max  min   sum  median       var       cov  \
0            NaN       NaN   NaN  NaN   NaN     NaN       NaN       NaN   
1            NaN       NaN   NaN  NaN   NaN     NaN       NaN       NaN   
2       8.666667  3.055050  12.0  6.0  26.0     8.0  9.333333  9.333333   
3       6.333333  1.527525   8.0  5.0  19.0     6.0  2.333333  2.333333   
4       7.000000  1.732051   8.0  5.0  21.0     8.0  3.000000  3.000000   
...          ...       ...   ...  ...   ...     ...       ...       ...   
299995  4.666667  0.577350   5.0  4.0  14.0     5.0  0.333333  0.333333   
299996  5.666667  1.154701   7.0  5.0  17.0     5.0  1.333333  1.333333   
299997  6.333333  1.154701   7.0  5.0  19.0     7.0  1.333333  1.333333   
299998  7.000000  0.000000   7.0  7.0  21.0     7.0  0.000000  0.000000   
299999  6.333333  1.154701   7.0  5.0  19.0     7.0  1.333333  1.333333   

        quantile_1  quantile_2  quantile_3  stepsize  
0              NaN         NaN         NaN  

In [88]:
X_train = X_train.fillna(method='backfill',axis=1)
X_train = X_train.fillna(method='ffill',axis=1)
print(X_train)

             ave       std   max  min   sum  median       var       cov  \
0      -6.000000 -6.000000  -6.0 -6.0  -6.0    -6.0 -6.000000 -6.000000   
1       2.000000  2.000000   2.0  2.0   2.0     2.0  2.000000  2.000000   
2       8.666667  3.055050  12.0  6.0  26.0     8.0  9.333333  9.333333   
3       6.333333  1.527525   8.0  5.0  19.0     6.0  2.333333  2.333333   
4       7.000000  1.732051   8.0  5.0  21.0     8.0  3.000000  3.000000   
...          ...       ...   ...  ...   ...     ...       ...       ...   
299995  4.666667  0.577350   5.0  4.0  14.0     5.0  0.333333  0.333333   
299996  5.666667  1.154701   7.0  5.0  17.0     5.0  1.333333  1.333333   
299997  6.333333  1.154701   7.0  5.0  19.0     7.0  1.333333  1.333333   
299998  7.000000  0.000000   7.0  7.0  21.0     7.0  0.000000  0.000000   
299999  6.333333  1.154701   7.0  5.0  19.0     7.0  1.333333  1.333333   

        quantile_1  quantile_2  quantile_3  stepsize  
0             -6.0        -6.0        -6.0  

In [92]:
kfold=KFold(n_splits=5,random_state=2020)

depth_range = [18]
minsplit_range = [20,25]
minleaf_range = [2,3,5]

parameters = dict(min_samples_split=minsplit_range, 
                  min_samples_leaf=minleaf_range)
rfr = RandomForestRegressor(oob_score=True,random_state=2020,max_depth=18)

cv = GridSearchCV(rfr,scoring='neg_mean_absolute_error',param_grid=parameters,cv=kfold,verbose=3)

y_train = y_train['time_to_failure']
type(y_train)

F:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


pandas.core.series.Series

In [93]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] min_samples_leaf=2, min_samples_split=20 ........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  min_samples_leaf=2, min_samples_split=20, score=-0.039, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split=20 ........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  min_samples_leaf=2, min_samples_split=20, score=-0.019, total= 1.1min
[CV] min_samples_leaf=2, min_samples_split=20 ........................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.3min remaining:    0.0s


[CV]  min_samples_leaf=2, min_samples_split=20, score=-0.005, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split=20 ........................
[CV]  min_samples_leaf=2, min_samples_split=20, score=-0.020, total= 1.1min
[CV] min_samples_leaf=2, min_samples_split=20 ........................
[CV]  min_samples_leaf=2, min_samples_split=20, score=-0.039, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split=25 ........................
[CV]  min_samples_leaf=2, min_samples_split=25, score=-0.039, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split=25 ........................
[CV]  min_samples_leaf=2, min_samples_split=25, score=-0.019, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split=25 ........................
[CV]  min_samples_leaf=2, min_samples_split=25, score=-0.005, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split=25 ........................
[CV]  min_samples_leaf=2, min_samples_split=25, score=-0.019, total= 1.2min
[CV] min_samples_leaf=2, min_samples_split

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 34.7min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2020, shuffle=False),
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=18,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=True, random_state=2020,
                                             verbo

In [94]:
cv.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=18, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=25, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=2020, verbose=0, warm_start=False)

In [95]:
cv.best_score_

-0.024137364235548225

In [96]:
cv.best_params_

{'min_samples_leaf': 2, 'min_samples_split': 25}

In [101]:
X_train.to_csv(r"C:\Users\dbda\Desktop\project\x_train.csv",index=False)
y_train.to_csv(r"C:\Users\dbda\Desktop\project\y_train.csv",index=False)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


Xgb